# 导入

In [2]:
import pandas as pd
import numpy as np

# 纳排标准：肺炎，＞18岁，1次ICU

In [3]:
# 细菌并且去除病毒感染的
diagnoses = pd.read_csv("mimic-iv-2/hosp/diagnoses_icd.csv", header=0)
diagnoses_9e = diagnoses.loc[diagnoses["icd_code"].str.match("48[1,2,3]\d"), :]
print(diagnoses_9e.shape)
diagnoses_10e = diagnoses.loc[diagnoses["icd_code"].str.match("J1[3,4,5]\d"), :]
print(diagnoses_10e.shape)
diagnoses_pn = pd.concat((diagnoses_9e, diagnoses_10e),0)
diagnoses_pn.to_csv("pn_bact/sofa_initial_diagnoses.csv", header=True, index=True)

(2915, 5)
(1671, 5)


/tmp/ipykernel_1396175/2094161175.py:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  diagnoses_pn = pd.concat((diagnoses_9e, diagnoses_10e),0)


In [4]:
diagnoses_virus = pd.read_csv("pn_virus/sofa_initial_diagnoses.csv")
diagnoses_bpn = diagnoses_pn.loc[~diagnoses_pn["hadm_id"].isin(set(diagnoses_virus["hadm_id"])),:]
diagnoses_bpn.to_csv("pn_bact/sofa_diagnoses.csv", header=True, index=True)
print(diagnoses_bpn.shape)

(4349, 5)


In [5]:
#  获取肺炎患者的stay信息并去除一次入院多次入住ICU的
patients = diagnoses_pn[["subject_id", "hadm_id"]].drop_duplicates()
icustay = pd.read_csv("mimic-iv-2/icu/icustays.csv", header=0)
patients_aicu = pd.merge(patients, icustay[["hadm_id","stay_id"]])
aicu_hadm = patients_aicu["hadm_id"].value_counts()[patients_aicu["hadm_id"].value_counts()==1]
patients_aicu =  patients_aicu.loc[patients_aicu["hadm_id"].isin(aicu_hadm.index),:]
patients_aicu.to_csv("pn_bact/patients_id.csv")

# 30天死亡分组

In [3]:
patients_id = pd.read_csv("pn_bact/patients_id.csv", header=0, index_col=0)
admission = pd.read_csv("mimic-iv-2/hosp/admissions.csv", header=0, index_col=0)
patients_oc = pd.merge(patients_id, admission[["hadm_id", "deathtime"]])

In [4]:
# icu stay
icustay = pd.read_csv("mimic-iv-2/icu/icustays.csv", header=0, index_col=0)
patients_icu_oc = pd.merge(patients_oc, icustay[["stay_id","intime"]])

# 计算入住ICU时间和死亡时间差
from datetime import datetime
death_days = []
for i in patients_icu_oc.index:
    if str(patients_icu_oc.loc[i,"deathtime"]) != "nan":
        death_day = (datetime.strptime(patients_icu_oc.loc[i,"deathtime"], "%Y-%m-%d %H:%M:%S") - 
                     datetime.strptime(patients_icu_oc.loc[i,"intime"], "%Y-%m-%d %H:%M:%S")).days
        death_days.append(death_day)
    else:
        death_days.append(31)
patients_icu_oc["death_day"] = death_days

In [5]:
# 分别保存30天内死亡患者
patients_icu_oc["30day_death"] = 0
patients_icu_oc.loc[patients_icu_oc["death_day"]<=30, "30day_death"] = 1
patients_icu_oc.loc[patients_icu_oc["death_day"]>30, "30day_death"] = 0
patients_icu_oc.loc[patients_icu_oc["death_day"]>30, "death_day"] = 31

patients_death = patients_icu_oc.loc[patients_icu_oc["30day_death"]==1,:]
patients_survival = patients_icu_oc.loc[patients_icu_oc["30day_death"]==0,:]

patients_death.to_csv("pn_bact/patients_death.csv")
patients_survival.to_csv("pn_bact/patients_survival.csv")
patients_icu_oc.to_csv("pn_bact/patients_oc.csv")